This week, we will go through the derivation of ELBO and its common usage: EM algorithm implementation.

# 0. Jensen's inequality

One of the important prerequisite in ELBO is the famous Jensen's inequality. This is simple but powerful concept, derived from the concept of convex function.

Given a convex function $f(x)$ and for $t \in [0,1]$,

$$
f(tx_1 + (1-t)x_2) \le tf(x_1) + (1-t)f(x_2)
$$

This quite simple notion can be generalized to finite form. For a convex function $\phi$ and positive weights $a_i: \sum a_i = 1$,

$$
\phi \left(\frac{\sum a_i x_i}{\sum a_i} \right) \le \frac{\sum a_i \phi(x_i)}{\sum a_i}
$$

The finite form can be easily achieved by induction. From here, we can go one step further: infinite, measure-theoretic form.

For a probability space $(\Omega, A, \mu)$, let $f: \Omega \to \mathbb{R}$. We also have $\phi: \mathbb{R} \to \mathbb{R}$ as a convex function. Then,

$$
\phi \left(\int_{\Omega}f d\mu \right) \le \int_{\Omega} \phi \circ f d\mu
$$

which can be specifically applied to a probability density function.

Let's say we have a pdf $f(x)$ in that $\int_{-\infty}^{\infty}f(x) dx = 1$ and $f(x) \ge 0$. Given a convex function $\phi$ and any real-valued measurable function $g(x)$,

$$
\phi \left(\int_{-\infty}^{\infty}g(x)f(x) dx \right) \le \int_{-\infty}^{\infty}\phi(g(x))f(x) dx
$$

Especially, this leads to the expectation as we set $g(x) = x$:

$$
\phi \left(\int_{-\infty}^{\infty}xf(x) dx \right) \le \int_{-\infty}^{\infty}\phi(x)f(x) dx
$$

## 1. What brings us the problem?

Our world is tough to understand. Not everything we observe is as merely it is but the sheer outcome of a complex underlying (i.e. latent) dynamics. Understanding what is really happening under the hood should be very beneficial - namely, we can wrap this motivation to understand the world as a language of the variational inference.

We start by saying we have an observed dataset $X$. We also believe there is some underlying generation process that leads to $X$: let's call this $Z$. Then, it is a natural urge to approximate the function that takes $X$ as input and estimates $Z$.

Bayes theorem provides a nice way of doing it:

$$
p(z|x) = \frac{p(x|z)p(z)}{p(x)}
$$

...or, if you don't like the expression such as $p(z)$ or $p(x)$, we can assume a "common sense" $c$:

$$
p(z|x,c) = \frac{p(x|z,c)p(z|c)}{p(x|c)}
$$

...but for the sake of brevity, let me drop $c$ for the rest of the equation.

Surely, we call...
1. $p(z|x)$ as the $\textbf{posterior}$ distribution given the observation
2. $p(x|z)$ as the $\textbf{likelihood}$ of the observation given the latent factor
3. $p(z)$ as the $\textbf{prior}$ distribution
4. $p(x)$ as the marginal likelihood (as $p(x) = \int_Z p(x,z)$), or $\textbf{EVIDENCE}$

What do we usually have at the beginning? We definitely don't have the posterior. We typically start from the assumption of the prior and likelihood function. The common historical choice of this pair is a conjugate prior. Though, these days, you can simply let a neural network to generate likelihood output.

These two assumptions easily equips us with the joint distribution $p(x,z)$ - but the problem arises with the last item that we don't have at hands: evidence! People say evidence, or the marginal likelihood $p(x) = \int_Z p(x,z)$ is often intractable.

Now, for me it was not very intuitive at the beginning - why can't you just approximate the evidence $p(x)$ from your sample observations? I mean - with extremely simple problem such as dice rolling, estimating or calculating $p(x)$ shouldn't be a big deal. Hell, we can roll it ten thousand times and take the estimation.

However, we do not always have that large amount of data / simple task. In the prior and likelihood based approach (as these are what we have in hands), our $p(x,z)$ from the assumption may not be easily marginalizable. Like, if we chose $p(z)$ being Cauchy distribution, or if we simply take neural network, then we do not have a very good closed-form analytic solution for $\int_Z p(x,z)$.

That's why we resort to some approximate inference methods. MCMC is a good example of numerical approximation methods, but we'll chat about this later. Today, we will nicely replace our problem by introducing ELBO.

## 2. ELBO derivation

Let's simply divide-and-conquer. In the last section, we argued the problem in estimating the posterior distribution $p(z|x)$ is to find $p(x)$. So, why don't we try to find $p(x)$?

Thanks to Jensen's inequality we introduced earlier, we can do this:

$$
\begin{align}
\ln p(x) &= \ln \int p(x,z) dz \\

&= \ln \int p(x,z) \frac{q(z)}{q(z)} dz \\

&= \ln \mathbb{E}_{q(z)} \frac{p(x,z)}{q(z)} \\

&\ge \mathbb{E}_{q(z)} \ln \frac{p(x,z)}{q(z)} \\

&= \mathbb{E}_{q(z)} \left[ \ln p(x,z) - \ln q(z) \right]

\end{align}
$$

Here we introduced another distribution $q(z)$. At this moment, it is very interesting to see the last term becomes zero when $p(x,z) = q(z)$. Does this mean anything? Taking a step back, why did we introduce this $q(z)$? Is it a simple trick? What did we want from this?

Our motivation of $q(z)$ becomes much clearer when we re-aim our focus back to our posterior $p(z|x)$. We introduce $q(z)$ in a sheer hope that, over some updatem, we might be able to nicely approximate $p(z|x)$.

But, how do we measure "approximate nicely"?

#### 2.1. KL divergence

We can intuitively think of the distance between $p(z|x)$ and $q(z)$ (or, again, $p(z|x,c)$ and $q(z|c)$ with common sense). One of the nice concept that allows us to compare probability distributions is the KL divergence.

## 5. The Downside of VI and ELBO

Let’s break this down carefully because there are actually two different layers of “parametric” assumptions in Bayesian modeling:

Parametric Model Assumption (the Bayesian model):

We choose a prior 
𝑝
(
𝑧
)
p(z) (e.g., a Gaussian) and a likelihood 
𝑝
(
𝑥
∣
𝑧
)
p(x∣z) (e.g., another parametric family).
This step is common to both MCMC and VI, because it defines the joint 
𝑝
(
𝑥
,
𝑧
)
p(x,z).
In other words, we typically do have a “parametric” or at least a specified functional form for 
𝑝
(
𝑧
)
p(z) and 
𝑝
(
𝑥
∣
𝑧
)
p(x∣z). This is what we call the generative model.
Inference Method Assumption (approximating the posterior 
𝑝
(
𝑧
∣
𝑥
)
p(z∣x)):

MCMC: Does not require choosing a parametric family for the posterior. Instead, it directly samples from 
𝑝
(
𝑧
∣
𝑥
)
p(z∣x) (which is fully determined by the model 
𝑝
(
𝑥
,
𝑧
)
p(x,z)) by building a Markov chain that converges (in principle) to the exact posterior.
So MCMC does not parametrically approximate 
𝑝
(
𝑧
∣
𝑥
)
p(z∣x). It just draws (potentially infinitely many) samples from the true posterior, assuming you run it long enough and the chain mixes well.
Variational Inference (VI): Does require choosing a parametric family 
𝑞
𝜙
(
𝑧
)
q 
ϕ
​
 (z) to approximate 
𝑝
(
𝑧
∣
𝑥
)
p(z∣x). For example, a factorized Gaussian, a mixture of Gaussians, or a normalizing flow with parameters 
𝜙
ϕ. You then optimize 
𝜙
ϕ (via the ELBO) so that 
𝑞
𝜙
(
𝑧
)
q 
ϕ
​
 (z) is as close as possible to the true posterior in some divergence sense.
Therefore:

Both MCMC and VI share the same “parametric model”: they assume you have already picked a prior 
𝑝
(
𝑧
)
p(z) and likelihood 
𝑝
(
𝑥
∣
𝑧
)
p(x∣z). That is a modeling choice.
The difference is that MCMC attempts to sample from the true posterior without imposing a further parametric form on 
𝑝
(
𝑧
∣
𝑥
)
p(z∣x), while VI uses a separate parametric family 
𝑞
𝜙
(
𝑧
)
q 
ϕ
​
 (z) to approximate 
𝑝
(
𝑧
∣
𝑥
)
p(z∣x).
Hence:

MCMC:
Parametric model for prior/likelihood (the usual Bayesian model).
But posterior is not forced into a parametric form; you approximate it by sampling from the exact posterior.
VI:
Same parametric model for prior/likelihood.
Additionally picks a parametric family 
𝑞
𝜙
(
𝑧
)
q 
ϕ
​
 (z) for the posterior approximation. This can cause “variational bias” if 
𝑞
𝜙
(
𝑧
)
q 
ϕ
​
 (z) cannot capture all complexities of 
𝑝
(
𝑧
∣
𝑥
)
p(z∣x).
That is the essential distinction.